In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from arcs.generate import GraphGenerator

graph = GraphGenerator().from_file(
    filename='../app/data/scan_dft_data.json',
    temperature=248,
    pressure=20,
    max_reaction_length=5
)

/Users/badw/github-projects/arcs/arcs/generate.py:255: RuntimeWarning: overflow encountered in exp
  K = np.exp(


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx 
import numpy as np 
G = graph 

node_colours= []
node_sizes = []
for node in list(G.nodes):
    if isinstance(node,int):
        node_colours.append('black')
        node_sizes.append(5)
    else:
        node_colours.append('white')
        node_sizes.append(100)

edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
vmin = np.min(weights)
vmax = np.max(weights)
cmap = plt.cm.viridis

nx.draw_random(G,node_color=node_colours,node_size=node_sizes,edgelist=edges,edge_color=weights,width=0.1,edge_cmap=cmap,vmin=vmin,vmax=vmax)

In [ ]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations
import pandas as pd #
gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2O':30,'O2':10,'SO2':10,'H2S':10,'NO2':10}
    )#
t = Traversal(graph=graph)#
t.rank_small_reactions_higher = {'option':True,'by_coefficients':False}
random_compounds = t.get_weighted_random_compounds(gic)
print(random_compounds)
random_reactions = t.get_weighted_reaction_rankings(random_compounds)
random_reactions 
for k in random_reactions:
    print(graph.nodes()[k]['reaction']['reaction_string'])

['H2O', 'SO2', 'H2S', 'NO2']
2 O2 + 1 H2S + 1 SO2 + 2 NO2 = 2 NOHSO4
5 H2S + 2 NO2 + 2 NOHSO4 = 7 SO2 + 4 NH3
4 H2S + 4 SO2 + 8 NO2 = 1 S8 + 8 HNO3
1 H2S + 2 H2SO3 + 6 NO2 = 3 SO2 + 6 HNO2
1 H2S + 1 SO2 + 4 NO2 = 1 N2 + 2 NOHSO4
2 H2 + 2 SO2 + 1 N2 = 2 H2S + 2 NO2
1 O2 + 3 H2S + 2 NO2 = 3 SO2 + 2 NH3
4 H2S + 12 SO2 + 8 NO2 = 1 S8 + 8 NOHSO4
2 H2 + 1 SO2 + 2 NO2 = 1 H2S + 2 HNO3
6 H2S + 6 NO2 = 6 SO2 + 4 NH3 + 1 N2
1 H2S + 4 NO2 = 1 SO2 + 2 HNO2 + 2 NO
2 H2 + 3 SO2 + 2 NO2 = 1 H2S + 2 NOHSO4
2 O2 + 1 H2S + 2 NO2 = 1 SO2 + 2 HNO3
1 H2 + 2 H2S + 2 NO2 = 2 SO2 + 2 NH3
1 H2S + 6 NOHSO4 = 3 SO2 + 4 H2SO3 + 6 NO2
1 H2S + 3 SO2 + 6 HNO3 = 4 H2SO3 + 6 NO2
1 H2S + 4 NO2 = 1 SO2 + 2 HNO3 + 1 N2
7 H2S + 2 H2SO3 + 6 NO2 = 9 SO2 + 6 NH3
2 H2S + 6 NO2 = 2 SO2 + 4 HNO2 + 1 N2
24 H2S + 16 NO2 = 1 S8 + 16 SO2 + 16 NH3


In [ ]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    {'H2O':30,'O2':10,'SO2':10,'H2S':10,'NO2':0}
    #{'H2O':30,'O2':10,'SO2':10,'H2S':10,'NO2':10}
    #{'H2O':30,'SO2':10}
    )

t = Traversal(graph=graph)

t.rank_small_reactions_higher = {'option':True,'by_coefficients':False}

data = t.sample(initial_concentrations=gic,ncpus=10,nsamples=100,progress_bar=True)

{'option': True, 'by_coefficients': False}


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling() #use_markdown=True,use_latex=True)
pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)

In [ ]:
average_data = analysis.average_sampling(data)
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.round(2)

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

sns.set_style('darkgrid')
sns.set_palette('pastel')


norm = colors.Normalize(vmin=average_data['diff'].min(), vmax=average_data['diff'].max())
cmap = cm.RdBu  # or any other colormap

bar_colors = [cmap(norm(value)) for value in average_data['diff']]

average_data.plot.bar(y='diff',yerr='std',color=bar_colors)

effect of adding NO2

In [ ]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic(
    #{'H2O':30,'O2':10,'SO2':10,'H2S':10,'NO2':0}
    {'H2O':30,'O2':10,'SO2':10,'H2S':10,'NO2':10}
    #{'H2O':30,'SO2':10}
    )

t = Traversal(graph=graph)

t.rank_small_reactions_higher = {'option':True,'by_coefficients':True}
print(t.rank_small_reactions_higher)

data = t.sample(initial_concentrations=gic,ncpus=10,nsamples=1000,progress_bar=True)

In [ ]:
from arcs.analysis import AnalyseSampling
import pandas as pd 

analysis = AnalyseSampling() #use_markdown=True,use_latex=True)
pd.Series(analysis.reaction_statistics(data)).sort_values(ascending=False)

In [ ]:
average_data = analysis.average_sampling(data)
average_data = average_data.loc[~(average_data==0).all(axis=1)]
average_data.round(5)
average_data.sort_values(by='diff',inplace=True)

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

sns.set_style('darkgrid')
sns.set_palette('pastel')


norm = colors.Normalize(vmin=average_data['diff'].min(), vmax=average_data['diff'].max())
cmap = cm.RdBu  # or any other colormap

bar_colors = [cmap(norm(value)) for value in average_data['diff']]

average_data.plot.bar(y='diff',yerr='std',color=bar_colors)

testing

In [ ]:
reaction_statistics = analysis.reaction_statistics(data)

rs = {}
reactions = [graph.nodes()[x]['reaction'] for x in graph.nodes() if isinstance(x,int)]
for i,reaction in enumerate(reactions):
    if reaction['reaction_string'] in list(reaction_statistics):
        rs[i] = reaction 
        rs[i]['frequency'] = reaction_statistics[reaction['reaction_string']]

In [ ]:
import networkx as nx 

G = nx.MultiDiGraph(directed=True)

for i,reaction in rs.items():
    G.add_weighted_edges_from(
                [compound,str(i),1] for compound in reaction['reactants']
                ) #reactants -> reaction
    G.add_weighted_edges_from(
                [str(i),compound,1] for compound in reaction['products']
                ) #reaction -> products
            
frequency = {str(i):reaction['frequency'] for i,reaction in rs.items()}

nx.set_node_attributes(G,frequency,name='frequency')

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx 
import numpy as np 
import seaborn as sns 
sns.set_palette('pastel')

node_colours= []
node_sizes = []
for node in list(G.nodes):
    try:
        int(node)
        node_colours.append('Tab:Red')
        try:
            node_sizes.append(G.nodes()[node]['frequency'])
        except Exception as e :
            print(e)
    except Exception as e:
        node_colours.append('Tab:Blue')
        node_sizes.append(1000)

nx.draw_circular(G,node_color=node_colours,node_size=node_sizes,with_labels=True)

In [ ]:


def node_size_transform(node):
    node_sizes = {}
    for x,v in dict(G.adjacency()).items():
        try:
            int(x)
            node_sizes[x]= 1
        except Exception:
            node_sizes[x] = len(v)
    return(node_sizes[str(node)])


In [ ]:
from pyvis.network import Network 
import networkx as nx 
g = Network(height=800,width=800,notebook=False,directed=True,neighborhood_highlight=True)
g.from_nx(G,show_edge_weights=False,node_size_transf=node_size_transform)
g.save_graph('graph.html')

In [ ]:
g.from_nx?

In [ ]:
t.generate_chempy_eqsystem(146)

### Speed Test 

In [ ]:
from arcs.traversal import Traversal
from arcs.generate import GenerateInitialConcentrations

gic = GenerateInitialConcentrations(graph=graph).update_ic({'H2O':100,'H2':30,'O2':50,'H2S':100,'NO2':200,'N2':300})
t = Traversal(graph=graph)

In [ ]:
data = t.sample(initial_concentrations=gic,ncpus=1)

In [ ]:
data = t.sample(initial_concentrations=gic,ncpus=2)

In [ ]:
data = t.sample(initial_concentrations=gic,ncpus=4)

In [ ]:
data = t.sample(initial_concentrations=gic,ncpus=10)

In [ ]:
t.random_walk(gic)